In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
from tabGAN.tabGAN import TabGAN
from src import constants as const
import utils

imports_path = "src//imports.ipynb"
tabGAN_path = "tabGAN.py"
utils_path = "utils\\utils.ipynb"

%run "$imports_path"

In [3]:
if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in consts")

In [4]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [14]:
%load_ext line_profiler
n_epochs = 100
n_critic = 10
adam_lr = 0.0002
adam_beta1 = 0.5
noise_discrete_unif_max = 0

batch_size = 500

n_synthetic_datasets = 10
n_synthetic_datasets_existing = 0

retrain = False

tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = adam_lr, adam_beta1 = adam_beta1,
              quantile_transformation_int = True, quantile_rand_transformation = False,
             noise_discrete_unif_max = noise_discrete_unif_max,
                gumbel_temperature = 0.5)

from utils.tictoc import tic, toc

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
tic()
with tf.device('/CPU:0'):
    tg_qtr.train(100, restart_training=True, progress_bar=True)
toc()

In [16]:
tic()
%lprun -f tg_qtr.train_step_func tg_qtr.train(10, restart_training=True, progress_bar=True, tf_make_train_step_graph=False)
toc()

  0%|          | 0/10 [00:00<?, ?it/s]

Elapsed time: 634.700985 seconds.



In [ ]:
tf.config.list_physical_devices()

In [6]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)


@tf.function
def traceme(x):
    return model(x)
%load_ext tensorboard

In [56]:
tf.profiler.experimental.start('logdir', options = options)
## Train the model here
for i in range(10000):
    with tf.profiler.experimental.Trace('train', step_num=i, _r=0):
        traceme(tf.zeros((1, 28, 28, 1)))
tf.profiler.experimental.stop()

In [57]:
%tensorboard --logdir=logdir

Reusing TensorBoard on port 6006 (pid 19040), started 0:36:01 ago. (Use '!kill 19040' to kill it.)

In [9]:
tg_qtr = TabGAN(data_train, n_critic = n_critic, opt_lr = adam_lr, adam_beta1 = adam_beta1,
              quantile_transformation_int = True, quantile_rand_transformation = False,
             noise_discrete_unif_max = noise_discrete_unif_max,
                gumbel_temperature = 0.5)

In [7]:
%tensorboard --logdir=logTabGAN

In [10]:
tg_qtr.train(10, restart_training=True, progress_bar=True, tf_profile_train_step_range=[5,7], tf_profile_log_dir="logTabGAN")

  0%|          | 0/10 [00:00<?, ?it/s]